In [2]:
import pandas as pd
import numpy as np


X_train = pd.read_csv('X_train_prep_prosjekt.csv', sep=';')
y_train =  pd.read_csv('y_train_prep_prosjekt.csv', sep=';')
X_val = pd.read_csv('X_val_prep_prosjekt.csv', sep=';')
y_val = pd.read_csv('y_val_prep_prosjekt.csv', sep=';')
X_test = pd.read_csv('X_test_prep_prosjekt.csv', sep=';')
y_test = pd.read_csv('y_test_prep_prosjekt.csv', sep=';')

target = pd.read_csv('target_prep_prosjekt.csv', sep=';')

In [3]:

X_train

,Datetime,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Ukedag,Time,Helligdag,MånedGj
0,2015-07-16 16:00:00,48.7,13.733333,317.500000,4.333333,3,16,0.0,1386
1,2015-07-16 17:00:00,60.0,13.866667,318.166667,3.933333,3,17,0.0,1386
2,2015-07-16 18:00:00,60.0,13.216667,319.833333,4.233333,3,18,0.0,1386
3,2015-07-16 19:00:00,60.0,12.683333,323.500000,2.950000,3,19,0.0,1386
4,2015-07-16 20:00:00,36.0,12.066667,333.500000,2.483333,3,20,0.0,1386
...,...,...,...,...,...,...,...,...,...
38839,2019-12-31 19:00:00,0.0,5.350000,146.833333,2.016667,1,19,1.0,791
38840,2019-12-31 20:00:00,0.0,5.850000,146.000000,2.016667,1,20,1.0,791
38841,2019-12-31 21:00:00,0.0,5.866667,152.333333,2.783333,1,21,1.0,791
38842,2019-12-31 22:00:00,0.0,5.816667,141.666667,3.083333,1,22,1.0,791


In [4]:
data2022 = target.copy()
data2022

,Datetime,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Ukedag,Time,Helligdag,MånedGj
0,2021-12-31 23:00:00,0.0,3.483333,274.000000,1.233333,4,23,0.0,791
1,2022-01-01 00:00:00,0.0,3.016667,132.166667,0.683333,5,0,1.0,827
2,2022-01-01 01:00:00,0.0,2.666667,112.500000,1.233333,5,1,1.0,827
3,2022-01-01 02:00:00,0.0,2.316667,114.666667,1.233333,5,2,1.0,827
4,2022-01-01 03:00:00,0.0,1.733333,125.166667,0.900000,5,3,1.0,827
...,...,...,...,...,...,...,...,...,...
3550,2022-05-28 22:00:00,0.0,7.450000,343.333333,4.983333,5,22,0.0,2023
3551,2022-05-28 23:00:00,0.0,7.133333,344.000000,5.050000,5,23,0.0,2023
3552,2022-05-29 00:00:00,0.0,7.016667,346.833333,4.566667,6,0,0.0,2023
3553,2022-05-29 01:00:00,0.0,6.816667,348.000000,3.533333,6,1,0.0,2023


In [5]:
X_train = X_train.drop(['Datetime'], axis=1)
y_train = y_train.drop(['Datetime'], axis=1)
X_val = X_val.drop(['Datetime'], axis=1)
y_val = y_val.drop(['Datetime'], axis=1)
X_test = X_test.drop(['Datetime'], axis=1)
y_test = y_test.drop(['Datetime'], axis=1)
target = target.drop(['Datetime'], axis=1)

In [6]:
y_train['Volum'] = y_train['Volum'].astype(int) 
y_val['Volum'] = y_val['Volum'].astype(int) 
y_test['Volum'] = y_test['Volum'].astype(int) 

# Imputating, encoding, scaling

Må imputere de manglende verdiene. Gi en binær kode til de kategoriske datetimekolonnene slik at maskinen ikke regner 
fredag > torsdag.

In [7]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [8]:
num_cols = ['Solskinstid','Lufttemperatur','Vindretning','Vindstyrke', 'Helligdag', 'MånedGj'  ]

cat_cols = ['Ukedag','Time']

Ønsker å gjøre imputasjon, skalering og encoding så effektivt som mulig. Gjør dette ved hjelp av en kombinasjon av en Pipeline og en ColumnTransformer fra sklearn. Fordi jeg selv har laget alle kategoriske variabler, vet jeg at ingen av disse har manglende verdier, så de behøver ikke imputeres. Begynner derfor med å lage en Pipeline; num_pipeline, som imputerer alle manglende numeriske verdier med kNN-imputasjon, som er begrunnet i 2.2, for å deretter skalere alle numeriske verdier:




Kilde :https://www.youtube.com/watch?v=a6o9ies85eM


In [9]:
pp_num = Pipeline([
                        
                        ('imputer', KNNImputer(missing_values = np.nan, weights = 'distance')),
                        ('scaler', MinMaxScaler())
])



Integrerer denne Pipelinen i en ColumnTransformer; sei_transformer, som både imputerer og skalerer numeriske variabler, samtidig som den encoder kategoriske variabler ved hjelp av sklearns' OneHotEncoder:

De kategoriske variablene har ingen Nan-verdier og trenger dermed ikke å lage en egen pipeline da jeg skal bare utføre encoding.

In [10]:
ct = ColumnTransformer([
                                    ('pp_num', pp_num, num_cols),
                                    ('pp_cat', OneHotEncoder(), cat_cols)
])

In [11]:
X_train = ct.fit_transform(X_train)
X_val = ct.fit_transform(X_val)
X_test = ct.fit_transform(X_test)



In [12]:
target = ct.fit_transform(target)

In [13]:
pd.DataFrame(target)

,0
0,"(0, 1)\t0.2616459627329193\n (0, 2)\t0.7663..."
1,"(0, 1)\t0.23990683229813672\n (0, 2)\t0.357..."
2,"(0, 1)\t0.22360248447204972\n (0, 2)\t0.300..."
3,"(0, 1)\t0.20729813664596275\n (0, 2)\t0.306..."
4,"(0, 1)\t0.18012422360248453\n (0, 2)\t0.337..."
...,...
3550,"(0, 1)\t0.44642857142857156\n (0, 2)\t0.966..."
3551,"(0, 1)\t0.43167701863354047\n (0, 2)\t0.968..."
3552,"(0, 1)\t0.42624223602484484\n (0, 2)\t0.976..."
3553,"(0, 1)\t0.4169254658385094\n (0, 2)\t0.9798..."


In [14]:
X_train = (X_train).toarray()

X_val = (X_val).toarray()

X_test = (X_test).toarray()

target = (target).toarray()

# Evaluering av modeller



Tester tre fundamentalt ulike modeller på datasettet. Evaluerer ytelsen av systemet ved å sammenligne predikert antall sykklinger med kjent antall på validerings/test data settet. Sytemet blir evaluert med RMSE

Lager en metode (construct model), som vil konstruere modellen, trene med treningssettene, og evaluere modellene med RMSE.

modellene jeg velger å teste er Naive Bayes, K-Nearest Neighbour (KNN) og Random Forest Classifier

Jeg tester også sklearns dummy regressor for å skape en baseline. Dummy regressoren gjetter samme tall hver gang, og vil dermed ha mange feil.

Jeg forventer at både Decision Tree Classifier og KNN presterer bra. Naive _Bayes derimmot er bedre på å gjette binære verdier enn ordinale summer som vi leter etter nå, og jeg forventer dermed at den presterer ganske dårlig.

In [15]:
from sklearn.tree import DecisionTreeRegressor as tree
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsRegressor as knn
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

In [16]:
def evaluate_model(regressor):
    
    reg = regressor
    
    reg.fit(X_train, y_train)
    
    prediction = reg.predict(X_val)
    
    rmse = mean_squared_error(y_val, prediction, squared = False)
    
    print('RMSE:', rmse)
    

In [17]:
evaluate_model(DummyRegressor())

RMSE: 74.39309348924421


In [18]:
evaluate_model(tree())

RMSE: 41.09065721859091


In [19]:
evaluate_model(knn())

RMSE: 33.517997435627294


In [20]:
evaluate_model(GNB())

C:\Users\oivin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 152.28081908022753



K-Nearest neighbor regressor hadde lavest RMSE, nemlig lavest feil. KKN regresjon først identifiserer K-observasjoner som er nærmest prediksjonspunktet x_0, representert med N_0. Prediksjonene er deretter estimert med å ta gjennomsnittet av alle trenings responsene. Prediksjonene er dermed avhengig av hvor vi velger K til å være. En lav K vil ha et fleksibelt passområde med lavt partiskhet, men høy varians, og en høy K vil ha vice-versa. 

Velger K til å være 10, og ekstra vekt på datapunktene som er nærme,

In [21]:
model = knn(n_neighbors=10, weights = 'distance', algorithm='brute')

In [22]:
model.fit(X_train, y_train)

KNeighborsRegressor(algorithm='brute', n_neighbors=10, weights='distance')

In [26]:
k = model.predict(target)

In [27]:
k

array([[ 8.10609384],
       [ 3.60201584],
       [ 0.87050267],
       ...,
       [10.45402895],
       [ 4.68916626],
       [ 4.98685595]])

Gjør svarene til integers da volum originalt var int

In [28]:
k = k.astype(int)

In [29]:
predictions2022 = [element for innerList in k for element in innerList]

In [30]:
df = pd.DataFrame({'Volum':predictions2022})



In [31]:
df

,Volum
0,8
1,3
2,0
3,1
4,1
...,...
3550,14
3551,12
3552,10
3553,4


In [32]:
df = pd.concat([df,data2022],axis=1)


df = df.set_index(['Datetime'])

In [33]:
df

,Volum,Solskinstid,Lufttemperatur,Vindretning,Vindstyrke,Ukedag,Time,Helligdag,MånedGj
Datetime,,,,,,,,,
2021-12-31 23:00:00,8,0.0,3.483333,274.000000,1.233333,4,23,0.0,791
2022-01-01 00:00:00,3,0.0,3.016667,132.166667,0.683333,5,0,1.0,827
2022-01-01 01:00:00,0,0.0,2.666667,112.500000,1.233333,5,1,1.0,827
2022-01-01 02:00:00,1,0.0,2.316667,114.666667,1.233333,5,2,1.0,827
2022-01-01 03:00:00,1,0.0,1.733333,125.166667,0.900000,5,3,1.0,827
...,...,...,...,...,...,...,...,...,...
2022-05-28 22:00:00,14,0.0,7.450000,343.333333,4.983333,5,22,0.0,2023
2022-05-28 23:00:00,12,0.0,7.133333,344.000000,5.050000,5,23,0.0,2023
2022-05-29 00:00:00,10,0.0,7.016667,346.833333,4.566667,6,0,0.0,2023


In [ ]:
df.to_csv('predictions.csv', sep=';')